In [51]:
import os, torch, glob, torchvision, pathlib
import numpy as np
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable

# This program uses the non-cuda version of Torch, meaning all processes run on the CPU rather than the GPU.

In [52]:
transformer = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [53]:
trainpath = "/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Trainpath"
testpath = "/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Testpath"

trainset = DataLoader(torchvision.datasets.ImageFolder(trainpath, transform = transformer), batch_size = 3, shuffle = True)
testset = DataLoader(torchvision.datasets.ImageFolder(testpath, transform = transformer), batch_size = 3, shuffle = True)

In [54]:
rootdir = pathlib.Path(trainpath)
classes = sorted([j.name.split("/")[-1] for j in rootdir.iterdir()])
print(classes)

['no', 'yes']


In [60]:
class Network(nn.Module):
    def __init__(self, num_classes = 2):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 3, stride = 1, padding = 1)
        self.bn1 = nn.BatchNorm2d(num_features = 12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size = 2)
        
        self.conv2 = nn.Conv2d(in_channels = 12, out_channels = 20, kernel_size = 3, stride = 1, padding = 1)
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels = 20, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.bn3 = nn.BatchNorm2d(num_features = 32)
        self.relu3 = nn.ReLU()
        
        self.fc = nn.Linear(in_features = 32*50*50, out_features = num_classes)
        
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        
        output = self.pool(output)
        
        output = self.conv2(output)
        output = self.relu2(output)
        
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        
        output = output.view(-1, 32*50*50)
        output = self.fc(output)
        
        return output        

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [62]:
model = Network(num_classes = 2).to(device)
optimizer = Adam(model.parameters(), lr = 0.001, weight_decay = 0.0001)
loss_function = nn.CrossEntropyLoss()

In [63]:
num_epochs = 20
train_count = len(glob.glob(trainpath + "/**/*.jpg"))
test_count = len(glob.glob(testpath + "/**/*.jpg"))

print(train_count, test_count)

171 171


In [64]:
best_accuracy = 0.0
for epochs in range(num_epochs):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    test_accuracy = 0.0
    
    for i, (images, labels) in enumerate(testset):
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.cpu().data*images.size(0)
        _,prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count
    
    model.eval()
    
    for i, (images, labels) in enumerate(testset):
        outputs = model(images)
        _,predictions = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
    
    test_accuracy = test_accuracy/test_count
    
    print("Epochs: " + str(epochs) + " Train Loss: " + str((train_loss)) + " Training Accuracy: " + str(train_accuracy) + " Testing Accuracy: " + str(test_accuracy))
    
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), "best_checkpoint.model")
        best_accuracy = test_accuracy

Epochs: 0 Train Loss: tensor(10.3107) Training Accuracy: 0.9941520467836257 Testing Accuracy: 0.5730994152046783
Epochs: 1 Train Loss: tensor(7.1981) Training Accuracy: 1.0994152046783625 Testing Accuracy: 0.9064327485380117
Epochs: 2 Train Loss: tensor(3.2173) Training Accuracy: 1.2456140350877194 Testing Accuracy: 0.9064327485380117
Epochs: 3 Train Loss: tensor(3.0053) Training Accuracy: 1.2514619883040936 Testing Accuracy: 0.5730994152046783
Epochs: 4 Train Loss: tensor(0.4818) Training Accuracy: 1.4210526315789473 Testing Accuracy: 0.9064327485380117
Epochs: 5 Train Loss: tensor(1.1741) Training Accuracy: 1.368421052631579 Testing Accuracy: 0.9064327485380117
Epochs: 6 Train Loss: tensor(2.2151) Training Accuracy: 1.3333333333333333 Testing Accuracy: 0.5730994152046783
Epochs: 7 Train Loss: tensor(0.4831) Training Accuracy: 1.432748538011696 Testing Accuracy: 0.9064327485380117
Epochs: 8 Train Loss: tensor(0.4446) Training Accuracy: 1.432748538011696 Testing Accuracy: 0.57309941520